In [1]:
!pip install jovian opendatasets matplotlib seaborn xgboost --upgrade --quiet

In [2]:
# Import packages
import opendatasets as od
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

from datetime import datetime

## Downloading the Data
The dataset is obtained from kaggle. The dataset contains information on historic trades for several cryptoassets, such as Bitcoin and Ethereum.

In [3]:
data_url = 'https://www.kaggle.com/c/g-research-crypto-forecasting/data'
od.download(data_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

In [ ]:
data_dir = './g-research-crypto-forecasting'


In [ ]:
os.listdir(data_dir)


## Read in Dataset

In [ ]:
train_df = pd.read_csv('./g-research-crypto-forecasting/train.csv')
asset_details_df = pd.read_csv('./g-research-crypto-forecasting/asset_details.csv')
test_df = pd.read_csv('./g-research-crypto-forecasting/example_test.csv')

In [ ]:
train_df.head()

## Data Preparation and Cleaning
The training set has the following variables

timestamp - A timestamp for the minute covered by the row.

Asset_ID - An ID code for the cryptoasset.

Count - The number of trades that took place this minute.

Open - The USD price at the beginning of the minute.

High - The highest USD price during the minute.

Low - The lowest USD price during the minute.

Close - The USD price at the end of the minute.

Volume - The number of cryptoasset units traded during the minute.

VWAP - The volume weighted average price for the minute.

Target - 15 minute residualized returns.

The following variables will be created based on the timestamp column.

hour is the hour of the day

weekday is the weekday of the week

Categorical variables (Asset_ID, hour, and weekday) will be on hot encoded.

Numeric variables (Count, Open, High, Low, Close, Volume, and VWAP) will be scaled to the value of 0 to 1 for each of the Asset_ID.

In [ ]:
train_df.shape

The training data has 24 million records and 10 columns

In order to reduce the data for lighter manipulation we take a subset up to November 2020

In [ ]:
train_df

The reduced training set has 598769 records.

## Handeling Missing Values


In [ ]:
#train_df.reindex(range(train_df.index[0],train_df.index[-1]+60,60),method='pad')

The data is registered every 60 s, but gaps appear. The first thing we do is to complete the time gaps by using the previous valid data for each crypto asset.

In [ ]:
for i in range(0,14):
    train_df[train_df["Asset_ID"] ==i].reindex(range(train_df[train_df["Asset_ID"] ==i].index[0],train_df[train_df["Asset_ID"] ==i].index[-1]+60,60),method='pad')


We check for missing features

In [ ]:
train_df.isna().sum()


In [ ]:
train_df.shape

In [ ]:
750338/24236806

In [ ]:
train_df.isin([np.nan, np.inf, -np.inf]).sum()


There is no infinity value in the data

The percentage of data without a Target value is low so we decide to ignore them. The following code will drop the missing values and infinity values.

In [ ]:
train_df.dropna(inplace=True)
train_df = train_df[np.isfinite(train_df).all(1)]

In [ ]:
train_df.describe()


## Feature Engineering
The following code converts timestamp to date time type and creates variables hour and weekday based on the value of column timestamp.

In [ ]:
# Convert timestamp to date time
train_df['timestamp'] = train_df.timestamp.astype('datetime64[s]')
train_df = train_df[train_df.timestamp.dt.year==2020]
train_df = train_df[train_df.timestamp.dt.month==11]
train_df.shape

In [ ]:
# Create hour variable
train_df['hour'] = train_df.timestamp.dt.hour

# Create weekday variable
train_df['weekday'] = train_df.timestamp.dt.weekday

### Encode Categorical Variables
This section will convert variables Asset_ID, hour and weekday to categorical variables and create one hot encoder for categorical variables

In [ ]:
# Convert Asset_ID, hour and weekday to categorical
train_df['Asset_ID'] = train_df.Asset_ID.astype('category')
train_df['hour'] = train_df.hour.astype('category')
train_df['weekday'] = train_df.weekday.astype('category')

In [ ]:
# Set up Encoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# fit onehotcoder
encoder.fit(train_df[['Asset_ID','hour','weekday']])

In [ ]:
encoder.__dict__

In [ ]:
# get new encoded cols names
encoded_cols = list(encoder.get_feature_names(['Asset_ID','hour','weekday']))

# replace categorical variables with one hot encoder
train_df[encoded_cols] = encoder.transform(train_df[['Asset_ID','hour','weekday']])

### Scale Numeric Variables
This section will scale numeric variables Count, Open, High, Low, Close, Volume, and VWAP to range from 0 to 1.

In [ ]:
scaler = MinMaxScaler()


num_cols = ['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']

scaler.fit(train_df[num_cols])
train_df[num_cols] = scaler.transform(train_df[num_cols])

In [ ]:
train_df.head()

## Time Series Plots
The following code get the time range for each of the Asset_ID.

In [ ]:
train_df.groupby(["Asset_ID"]).agg({'timestamp': [np.min,np.max]})


The date range is slightly different for each of the asset.



In [ ]:
plt.figure(figsize=(50, 25))
sns.lineplot(data=train_df, x="timestamp", y="Target", hue="Asset_ID")

The Target variable has very different patterns for each of the asset of interest.

#### Correlation Matrix
The following code provide correlation matrix for each of the asset.

In [ ]:
corr_matrix = train_df[['Count','Open','High','Low','Close','Volume','VWAP','Target']].corr()
plt.figure(figsize=(10, 10))
sns.heatmap(corr_matrix,annot=True, cmap='Blues');

The Open, High, Low, Close, and VWAP are highly correlated.

### Training and Testing Split

Use data from November 1st - November 20th as training set, use the rest data in November as validation set. The testing set is provided in example_test.csv file.

In [ ]:
train_set = train_df[train_df.timestamp.dt.day <= 20]
val_set = train_df[train_df.timestamp.dt.day > 20]

In [ ]:
train_set.shape


In [ ]:
val_set.shape


In [ ]:
train_set.Asset_ID.value_counts()


In [ ]:
val_set.Asset_ID.value_counts()


### Random Forest

This section build random forest to predict the Target variable.

In [ ]:
# Create the model
rf1 = RandomForestRegressor(random_state=5, n_jobs = -1)

In [ ]:
# Fit the model
input_cols = num_cols + encoded_cols
target_col = 'Target'

rf1.fit(train_set[input_cols], train_set[target_col])

In [ ]:
# Model Evaluation
tree_train_preds = rf1.predict(train_set[input_cols])
tree_val_preds = rf1.predict(val_set[input_cols])

tree_train_rmse = mean_squared_error(train_set[target_col], tree_train_preds, squared=False)
tree_val_rmse = mean_squared_error(val_set[target_col], tree_val_preds, squared=False)

print("Train Error is ", tree_train_rmse)
print("Validation Error is ", tree_val_rmse)

In [ ]:
# functions for hyperparameter tunning
def test_params(**params):
    model = RandomForestRegressor(random_state=5, n_jobs=-1, **params).fit(train_set[input_cols], train_set[target_col])
    train_rmse = mean_squared_error(model.predict(train_set[input_cols]), train_set[target_col], squared=False)
    val_rmse = mean_squared_error(model.predict(val_set[input_cols]), val_set[target_col], squared=False)
    return train_rmse, val_rmse

def test_param_and_plot(param_name, param_values):
    train_errors, val_errors = [], [] 
    for value in param_values:
        params = {param_name: value}
        train_rmse, val_rmse = test_params(**params)
        train_errors.append(train_rmse)
        val_errors.append(val_rmse)
    plt.figure(figsize=(10,6))
    plt.title('Overfitting curve: ' + param_name)
    plt.plot(param_values, train_errors, 'b-o')
    plt.plot(param_values, val_errors, 'r-o')
    plt.xlabel(param_name)
    plt.ylabel('RMSE')
    plt.legend(['Training', 'Validation'])

In [ ]:
# max_depth
test_param_and_plot('max_depth', [5, 10, 15, 20, 25, 30, 35])

In [ ]:
# n_estimators
test_param_and_plot('n_estimators', [5, 10, 50, 100])

In [ ]:
rf2 = RandomForestRegressor(random_state=5, n_jobs = -1, max_depth=5, n_estimators=5)

rf2.fit(train_set[input_cols], train_set[target_col])

tree_train_preds = rf2.predict(train_set[input_cols])
tree_val_preds = rf2.predict(val_set[input_cols])

tree_train_rmse = mean_squared_error(train_set[target_col], tree_train_preds, squared=False)
tree_val_rmse = mean_squared_error(val_set[target_col], tree_val_preds, squared=False)

print("Train Error is ", tree_train_rmse)
print("Validation Error is ", tree_val_rmse)

In [ ]:
rf2_importance_df = pd.DataFrame({
    'feature': train_set[input_cols].columns,
    'importance': rf2.feature_importances_
}).sort_values('importance', ascending=False)

In [ ]:
sns.barplot(data=rf2_importance_df.head(10), x='importance', y='feature')


## Gradient Boost

In [ ]:
# Create the model
GB = XGBRegressor(random_state=5, n_jobs=-1, n_estimators=20, max_depth=4)

In [ ]:
# Fit the model
GB.fit(train_set[input_cols], train_set[target_col])

In [ ]:
# Model Evaluation
GB_train_preds = GB.predict(train_set[input_cols])
GB_val_preds = GB.predict(val_set[input_cols])

GB_train_rmse = mean_squared_error(train_set[target_col], GB_train_preds, squared=False)
GB_val_rmse = mean_squared_error(val_set[target_col], GB_val_preds, squared=False)

print("Train Error is ", GB_train_rmse)
print("Validation Error is ", GB_val_rmse)

In [ ]:
# functions for hyperparameter tunning
def test_params(**params):
    model = XGBRegressor(random_state=5, n_jobs=-1, **params).fit(train_set[input_cols], train_set[target_col])
    train_rmse = mean_squared_error(model.predict(train_set[input_cols]), train_set[target_col], squared=False)
    val_rmse = mean_squared_error(model.predict(val_set[input_cols]), val_set[target_col], squared=False)
    return train_rmse, val_rmse

def test_param_and_plot(param_name, param_values):
    train_errors, val_errors = [], [] 
    for value in param_values:
        params = {param_name: value}
        train_rmse, val_rmse = test_params(**params)
        train_errors.append(train_rmse)
        val_errors.append(val_rmse)
    plt.figure(figsize=(10,6))
    plt.title('Overfitting curve: ' + param_name)
    plt.plot(param_values, train_errors, 'b-o')
    plt.plot(param_values, val_errors, 'r-o')
    plt.xlabel(param_name)
    plt.ylabel('RMSE')
    plt.legend(['Training', 'Validation'])

In [ ]:
# n_estimators
test_param_and_plot('n_estimators', [5, 10, 20, 40])

In [ ]:
# tuned model
GB2 = XGBRegressor(random_state=5, n_jobs=-1, n_estimators=20, max_depth=5)

GB2.fit(train_set[input_cols], train_set[target_col])

GB_train_preds = GB2.predict(train_set[input_cols])
GB_val_preds = GB2.predict(val_set[input_cols])

GB_train_rmse = mean_squared_error(train_set[target_col], GB_train_preds, squared=False)
GB_val_rmse = mean_squared_error(val_set[target_col], GB_val_preds, squared=False)

print("Train Error is ", GB_train_rmse)
print("Validation Error is ", GB_val_rmse)